In [8]:
# -*- coding: utf-8 -*-
import pandas as pd
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import os

# 1. 读取数据
file_path = os.path.join('.', 'data', 'wlw_voice2text_res.csv')
df = pd.read_csv(file_path, sep="#", names=["id", "text"], dtype=str)

df.head(5)


,id,text
0,asr_log_id,asr_text_res
1,964267663,喂你好，.您好很高兴为您服务请问有什么可以帮您.那个我我.不是这张我这张这张卡怎么就是没没流...
2,964267664,您好很高兴为您服务请问有什么可以帮您.我咨询一下我那个.电脑怎么连接你们联通给我带那个猫能不...
3,964267665,您好很高兴为您服务请问有什么可以帮您.喂你好就是就是我这边之前那个弄了个物联卡.嗯能查到信息...
4,964267666,你好很高兴为您服务请问有什么可以帮您.一九五八，.喂.喂你好.嗯你好请讲.啊我这边有一个物联...


In [9]:
# 2. 中文分词
def chinese_tokenizer(text):
    return list(jieba.cut(text))

df["tokens"] = df["text"].apply(chinese_tokenizer)
df.head(5)

,id,text,tokens
0,asr_log_id,asr_text_res,"[asr, _, text, _, res]"
1,964267663,喂你好，.您好很高兴为您服务请问有什么可以帮您.那个我我.不是这张我这张这张卡怎么就是没没流...,"[喂, 你好, ，, ., 您好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, ..."
2,964267664,您好很高兴为您服务请问有什么可以帮您.我咨询一下我那个.电脑怎么连接你们联通给我带那个猫能不...,"[您好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, 帮, 您, ., 我, 咨..."
3,964267665,您好很高兴为您服务请问有什么可以帮您.喂你好就是就是我这边之前那个弄了个物联卡.嗯能查到信息...,"[您好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, 帮, 您, ., 喂, 你..."
4,964267666,你好很高兴为您服务请问有什么可以帮您.一九五八，.喂.喂你好.嗯你好请讲.啊我这边有一个物联...,"[你好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, 帮, 您, ., 一九五,..."


In [10]:
# 3. 提取关键词（TF-IDF & TextRank）
def extract_keywords(text, top_k=5):
    tfidf_keywords = jieba.analyse.extract_tags(text, topK=top_k)
    textrank_keywords = jieba.analyse.textrank(text, topK=top_k)
    return {"TF-IDF": tfidf_keywords, "TextRank": textrank_keywords}

df["keywords"] = df["text"].apply(extract_keywords)
df.head(5)

,id,text,tokens,keywords
0,asr_log_id,asr_text_res,"[asr, _, text, _, res]","{'TF-IDF': ['asr', 'text', 'res'], 'TextRank':..."
1,964267663,喂你好，.您好很高兴为您服务请问有什么可以帮您.那个我我.不是这张我这张这张卡怎么就是没没流...,"[喂, 你好, ，, ., 您好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, ...","{'TF-IDF': ['一下', '商家', '流量', '这边', '张卡'], 'Te..."
2,964267664,您好很高兴为您服务请问有什么可以帮您.我咨询一下我那个.电脑怎么连接你们联通给我带那个猫能不...,"[您好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, 帮, 您, ., 我, 咨...","{'TF-IDF': ['宽带', '咨询', '10010', '10019', '专线'..."
3,964267665,您好很高兴为您服务请问有什么可以帮您.喂你好就是就是我这边之前那个弄了个物联卡.嗯能查到信息...,"[您好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, 帮, 您, ., 喂, 你...","{'TF-IDF': ['联网', '那个', '号码', '一下', '联通'], 'Te..."
4,964267666,你好很高兴为您服务请问有什么可以帮您.一九五八，.喂.喂你好.嗯你好请讲.啊我这边有一个物联...,"[你好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, 帮, 您, ., 一九五,...","{'TF-IDF': ['张卡', '你好', '卡号', '这边', '四六'], 'Te..."


In [11]:
# 4. 摘要提取（简单版：取TextRank前3关键词拼接）
df["summary"] = df["keywords"].apply(lambda x: " ".join(x["TextRank"]))
df.head(5)

,id,text,tokens,keywords,summary
0,asr_log_id,asr_text_res,"[asr, _, text, _, res]","{'TF-IDF': ['asr', 'text', 'res'], 'TextRank':...",
1,964267663,喂你好，.您好很高兴为您服务请问有什么可以帮您.那个我我.不是这张我这张这张卡怎么就是没没流...,"[喂, 你好, ，, ., 您好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, ...","{'TF-IDF': ['一下', '商家', '流量', '这边', '张卡'], 'Te...",商家 售后 个人用户 没有 显示
2,964267664,您好很高兴为您服务请问有什么可以帮您.我咨询一下我那个.电脑怎么连接你们联通给我带那个猫能不...,"[您好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, 帮, 您, ., 我, 咨...","{'TF-IDF': ['宽带', '咨询', '10010', '10019', '专线'...",宽带 专线 个人 选错 连接
3,964267665,您好很高兴为您服务请问有什么可以帮您.喂你好就是就是我这边之前那个弄了个物联卡.嗯能查到信息...,"[您好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, 帮, 您, ., 喂, 你...","{'TF-IDF': ['联网', '那个', '号码', '一下', '联通'], 'Te...",请问 没有 号码 办卡 系统
4,964267666,你好很高兴为您服务请问有什么可以帮您.一九五八，.喂.喂你好.嗯你好请讲.啊我这边有一个物联...,"[你好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, 帮, 您, ., 一九五,...","{'TF-IDF': ['张卡', '你好', '卡号', '这边', '四六'], 'Te...",不能 问题 工作人员 知道 来电


In [12]:
# 5. 简单问题分类（示例：关键词匹配）
def classify_issue(text):
    product_kw = ["设备", "硬件", "模块", "终端", "传感器"]
    service_kw = ["服务", "客服", "维修", "安装", "处理", "态度"]
    order_kw = ["订单", "发票", "物流", "收货", "付款"]

    if any(k in text for k in product_kw):
        return "产品问题"
    elif any(k in text for k in service_kw):
        return "服务问题"
    elif any(k in text for k in order_kw):
        return "订单问题"
    else:
        return "其他"

df["category"] = df["text"].apply(classify_issue)
df.head(5)

,id,text,tokens,keywords,summary,category
0,asr_log_id,asr_text_res,"[asr, _, text, _, res]","{'TF-IDF': ['asr', 'text', 'res'], 'TextRank':...",,其他
1,964267663,喂你好，.您好很高兴为您服务请问有什么可以帮您.那个我我.不是这张我这张这张卡怎么就是没没流...,"[喂, 你好, ，, ., 您好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, ...","{'TF-IDF': ['一下', '商家', '流量', '这边', '张卡'], 'Te...",商家 售后 个人用户 没有 显示,服务问题
2,964267664,您好很高兴为您服务请问有什么可以帮您.我咨询一下我那个.电脑怎么连接你们联通给我带那个猫能不...,"[您好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, 帮, 您, ., 我, 咨...","{'TF-IDF': ['宽带', '咨询', '10010', '10019', '专线'...",宽带 专线 个人 选错 连接,服务问题
3,964267665,您好很高兴为您服务请问有什么可以帮您.喂你好就是就是我这边之前那个弄了个物联卡.嗯能查到信息...,"[您好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, 帮, 您, ., 喂, 你...","{'TF-IDF': ['联网', '那个', '号码', '一下', '联通'], 'Te...",请问 没有 号码 办卡 系统,服务问题
4,964267666,你好很高兴为您服务请问有什么可以帮您.一九五八，.喂.喂你好.嗯你好请讲.啊我这边有一个物联...,"[你好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, 帮, 您, ., 一九五,...","{'TF-IDF': ['张卡', '你好', '卡号', '这边', '四六'], 'Te...",不能 问题 工作人员 知道 来电,服务问题


In [13]:
# 6. 输出结果
print(df.head(5))

# 7. 如果需要保存
df.to_csv(f"{os.path.join('.', 'data', 'wlw_voice2text_processed.csv')}", index=False, encoding="utf-8-sig")

           id                                               text  \
0  asr_log_id                                       asr_text_res   
1   964267663  喂你好，.您好很高兴为您服务请问有什么可以帮您.那个我我.不是这张我这张这张卡怎么就是没没流...   
2   964267664  您好很高兴为您服务请问有什么可以帮您.我咨询一下我那个.电脑怎么连接你们联通给我带那个猫能不...   
3   964267665  您好很高兴为您服务请问有什么可以帮您.喂你好就是就是我这边之前那个弄了个物联卡.嗯能查到信息...   
4   964267666  你好很高兴为您服务请问有什么可以帮您.一九五八，.喂.喂你好.嗯你好请讲.啊我这边有一个物联...   

                                              tokens  \
0                             [asr, _, text, _, res]   
1  [喂, 你好, ，, ., 您好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, ...   
2  [您好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, 帮, 您, ., 我, 咨...   
3  [您好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, 帮, 您, ., 喂, 你...   
4  [你好, 很, 高兴, 为您服务, 请问, 有, 什么, 可以, 帮, 您, ., 一九五,...   

                                            keywords           summary  \
0  {'TF-IDF': ['asr', 'text', 'res'], 'TextRank':...                     
1  {'TF-IDF': ['一下', '商家', '流量', '这边', '张卡'], 'Te...  商家 售后 个人用户 没有 显示   
2  {'TF-IDF': ['宽带', '咨询', '1001